In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sys
sys.path.append('.')

from poseestimation import model

pe = model.PoseEstimator()

img_path = './julian.JPG'

I = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

I_list = np.array([I, I])
print(I_list.shape)

heatmaps, pafs = pe.predict(I)

Using TensorFlow backend.
/home/julian/anaconda3/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(2, 1000, 750, 3)
(138, 184)
(276, 368)
(414, 552)
(552, 736)


In [4]:
pafs.shape

(1, 4, 1000, 750, 38)